# Random Forest, mesmo tamanho de vocabulário

In [1]:
import pandas as pd

df = pd.read_csv('selecionada_comparacao.txt', sep = '|', encoding = 'utf-8')
df.head(20)

,COD,NUM_ENUNCIADO,DESCR_AREA,PREPROC_ENUNCIADO,PREPROC_EXCERTO,COD_JURISPRUDENCIA,NUM,ANO,COD_COLEGIADO,PREPROC_INTEIRO_TEOR
0,1400,1236,Responsabilidade,comprovada participação beneficiário concessão...,votocuidam auto tomada conta especi instaurada...,1400,297.0,2016.0,P,tribun conta união tc 010084/2015-0 grupo ii -...
1,1700,1534,Finanças Públicas,operação ficta exportação plataforma petróleo ...,votocuidam auto solicitação congresso nacion r...,1700,366.0,2016.0,P,tribun conta união tc 005933/2014-5 grupo i - ...
2,5700,5314,Responsabilidade,âmbito tcu dosimetria pena balizador nível gra...,relatóriotrata-s embargo declaração oposto rec...,5700,944.0,2016.0,P,tribun conta união tc 042038/2012-0 grupo ii -...
3,284,40,Direito Processual,princípio independência instância permit tcu a...,voto8 relação outro processo judiciai tratam i...,284,30.0,2016.0,P,tribun conta união tc 000742/2014-7 grupo ii -...
4,298,54,Pessoal,é vedado dirigent entidad sistema s nomeação c...,voto11 relativament ato envolvendo sra cabe re...,298,55.0,2016.0,P,tribun conta união tc 013174/2012-6 grupo i - ...
5,299,55,Licitação,é irregular contratação emergenci dispensa lic...,voto2 present apelo têm objetivo contestar rej...,299,27.0,2016.0,P,tribun conta união tc 004063/2008-4 grupo i - ...
6,310,66,Pessoal,aplicação princípio segurança jurídica fin man...,votoexamina-s ato aposentadoria concedida âmbi...,310,38.0,2016.0,P,tribun conta união tc 005231/2012-4 grupo i - ...
7,364,120,Pessoal,redução provento aposentadoria concedida desac...,votoexamina-s ato aposentadoria concedida âmbi...,364,38.0,2016.0,P,tribun conta união tc 005231/2012-4 grupo i - ...
8,432,168,Licitação,indicação marca edit deve estar amparada razõ ...,vototrata-s representação formulada empresa re...,432,113.0,2016.0,P,tribun conta união tc 031921/2015-9 grupo i - ...
9,449,185,Licitação,permite-s menção marca referência edit forma p...,vototrata-s representação formulada empresa re...,449,113.0,2016.0,P,tribun conta união tc 031921/2015-9 grupo i - ...


In [2]:
import matplotlib.pyplot as plt

# Mostra gráfico de barras ilustando a distribuição da dimensão "Área" entre as 17 áreas existentes
df['DESCR_AREA'].value_counts().plot(kind='bar')
plt.show()

<Figure size 640x480 with 1 Axes>

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

textos_enunciados = df['PREPROC_ENUNCIADO'].values
textos_excertos = df['PREPROC_EXCERTO'].values
textos_acordaos = df['PREPROC_INTEIRO_TEOR'].values

def vetorizar(textos, nr_dimensoes):
    vetorizador = TfidfVectorizer(max_features=nr_dimensoes)
    vetorizador.fit(textos)
    return vetorizador.transform(textos)

MAX_FEATURES = 8000

textos_enunciados_bow = vetorizar(textos_enunciados, MAX_FEATURES)
textos_excertos_bow = vetorizar(textos_excertos, MAX_FEATURES)
textos_acordaos_bow = vetorizar(textos_acordaos, MAX_FEATURES)

print(textos_enunciados_bow.shape)
print(textos_excertos_bow.shape)
print(textos_acordaos_bow.shape)

(10486, 8000)
(10486, 8000)
(10486, 8000)


In [9]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
import warnings

warnings.filterwarnings('ignore')

kf = KFold(n_splits=10, random_state=42, shuffle=True)
f = 0

f1_macro_enunciados = []
f1_micro_enunciados = []
f1_weighted_enunciados = []
f1_classes_enunciados = []
f1_macro_excertos = []
f1_micro_excertos = []
f1_weighted_excertos = []
f1_classes_excertos = []
f1_macro_acordaos = []
f1_micro_acordaos = []
f1_weighted_acordaos = []
f1_classes_acordaos = []

for indice_treino, indice_teste in kf.split(df):
    
    df_treino = df.loc[indice_treino]
    df_teste = df.loc[indice_teste]
    
    textos_treino_enunciado = df_treino['PREPROC_ENUNCIADO'].values
    textos_teste_enunciado = df_teste['PREPROC_ENUNCIADO'].values
    textos_treino_excerto = df_treino['PREPROC_EXCERTO'].values
    textos_teste_excerto = df_teste['PREPROC_EXCERTO'].values
    textos_treino_acordao = df_treino['PREPROC_INTEIRO_TEOR'].values
    textos_teste_acordao = df_teste['PREPROC_INTEIRO_TEOR'].values
    
    textos_treino_enunciados_bow = textos_enunciados_bow[indice_treino]
    textos_teste_enunciados_bow = textos_enunciados_bow[indice_teste]
    textos_treino_excertos_bow = textos_excertos_bow[indice_treino]
    textos_teste_excertos_bow = textos_excertos_bow[indice_teste]
    textos_treino_acordaos_bow = textos_acordaos_bow[indice_treino]
    textos_teste_acordaos_bow = textos_acordaos_bow[indice_teste]
    

    print('Fold', f+1)

    print('\tTraining model (enunciados)...')
    rfm = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42)
    rfm.fit(textos_treino_enunciados_bow, df_treino['DESCR_AREA'])
    ps = rfm.predict(textos_teste_enunciados_bow)
    
    f1_macro_enunciados.append(f1_score(df_teste['DESCR_AREA'], ps, average='macro'))
    f1_micro_enunciados.append(f1_score(df_teste['DESCR_AREA'], ps, average='micro'))
    f1_weighted_enunciados.append(f1_score(df_teste['DESCR_AREA'], ps, average='weighted'))
    f1_classes_enunciados.append(f1_score(df_teste['DESCR_AREA'], ps, average=None))

    print('\tTraining model (excertos)...')
    rfm = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42)
    rfm.fit(textos_treino_excertos_bow, df_treino['DESCR_AREA'])
    ps = rfm.predict(textos_teste_excertos_bow)
    
    f1_macro_excertos.append(f1_score(df_teste['DESCR_AREA'], ps, average='macro'))
    f1_micro_excertos.append(f1_score(df_teste['DESCR_AREA'], ps, average='micro'))
    f1_weighted_excertos.append(f1_score(df_teste['DESCR_AREA'], ps, average='weighted'))
    f1_classes_excertos.append(f1_score(df_teste['DESCR_AREA'], ps, average=None))

    print('\tTraining model (acordaos)...')
    rfm = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42)
    rfm.fit(textos_treino_acordaos_bow, df_treino['DESCR_AREA'])
    ps = rfm.predict(textos_teste_acordaos_bow)
    
    f1_macro_acordaos.append(f1_score(df_teste['DESCR_AREA'], ps, average='macro'))
    f1_micro_acordaos.append(f1_score(df_teste['DESCR_AREA'], ps, average='micro'))
    f1_weighted_acordaos.append(f1_score(df_teste['DESCR_AREA'], ps, average='weighted'))
    f1_classes_acordaos.append(f1_score(df_teste['DESCR_AREA'], ps, average=None))

    f+=1

print('Fim')

Fold 1
	Training model (enunciados)...
	Training model (excertos)...
	Training model (acordaos)...
Fold 2
	Training model (enunciados)...
	Training model (excertos)...
	Training model (acordaos)...
Fold 3
	Training model (enunciados)...
	Training model (excertos)...
	Training model (acordaos)...
Fold 4
	Training model (enunciados)...
	Training model (excertos)...
	Training model (acordaos)...
Fold 5
	Training model (enunciados)...
	Training model (excertos)...
	Training model (acordaos)...
Fold 6
	Training model (enunciados)...
	Training model (excertos)...
	Training model (acordaos)...
Fold 7
	Training model (enunciados)...
	Training model (excertos)...
	Training model (acordaos)...
Fold 8
	Training model (enunciados)...
	Training model (excertos)...
	Training model (acordaos)...
Fold 9
	Training model (enunciados)...
	Training model (excertos)...
	Training model (acordaos)...
Fold 10
	Training model (enunciados)...
	Training model (excertos)...
	Training model (acordaos)...
Fim


In [11]:
import numpy as np

def percorre(lista1, lista2, lista3):
    for fold in range(kf.n_splits):
        print(fold+1, '\t', lista1[fold], '\t', lista2[fold], '\t', lista3[fold])
    print('média\t', np.mean(lista1), '\t', np.mean(lista2), '\t', np.mean(lista3))
    print('desvio', np.std(lista1), '\t', np.std(lista2), '\t', np.std(lista3))
        
def percorre2(lista1, lista2, lista3):
    for classe in range(len(rfm.classes_)):
        lt1 = []
        lt2 = []
        lt3 = []
        print('"' + rfm.classes_[classe] + '"  enunciados\t   excertos\t\t   acordaos')
        for fold in range(kf.n_splits):
            print(fold+1, '  ', lista1[fold][classe], '\t', lista2[fold][classe], '\t', lista3[fold][classe])
            lt1.append(lista1[fold][classe])
            lt2.append(lista2[fold][classe])
            lt3.append(lista3[fold][classe])
        print('média\t', np.mean(lt1), '\t', np.mean(lt2), '\t', np.mean(lt3))
        print('desvio', np.std(lt1), '\t', np.std(lt2), '\t', np.std(lt3))
    print()

        

print('f1_macro\t enunciados\t\t   excertos\t\t   acordaos')
percorre(f1_macro_enunciados, f1_macro_excertos, f1_macro_acordaos)
print()
print('f1_micro\t enunciados\t\t   excertos\t\t   acordaos')
percorre(f1_micro_enunciados, f1_micro_excertos, f1_micro_acordaos)
print()
print('f1_weighted\t enunciados\t\t   excertos\t\t   acordaos')
percorre(f1_weighted_enunciados, f1_weighted_excertos, f1_weighted_acordaos)
print()
print()
print('f1_classes:')
print()
percorre2(f1_classes_enunciados, f1_classes_excertos, f1_classes_acordaos)

f1_macro	 enunciados		   excertos		   acordaos
1 	 0.48851978755002934 	 0.5008978912325381 	 0.5019871914664231
2 	 0.4639018289368214 	 0.5189486891625256 	 0.4708752096078568
3 	 0.48798733044951736 	 0.5614088618834795 	 0.5054111423270814
4 	 0.4746310688349311 	 0.5042380467340664 	 0.42640663942503376
5 	 0.449172786437813 	 0.47100333521234256 	 0.4868444336595551
6 	 0.47938802558216437 	 0.5021360419784309 	 0.5064932244280747
7 	 0.4815483668843469 	 0.4935247412997256 	 0.443018533821187
8 	 0.4586417869240044 	 0.5328826339536489 	 0.4786904269209572
9 	 0.48915695388458386 	 0.5022299163389651 	 0.4751911794281634
10 	 0.4679661412734631 	 0.5404320309345491 	 0.5012760105770899
média	 0.4740914076757675 	 0.5127702188730272 	 0.47961939916614227
desvio 0.013088819669098594 	 0.02475150697314273 	 0.025840082598337193

f1_micro	 enunciados		   excertos		   acordaos
1 	 0.7445185891325071 	 0.7302192564346998 	 0.6320305052430887
2 	 0.7283126787416587 	 0.7244995233555768